# Quantum chemistry with a quantum computer

## Quantum chemistry

The bottleneck of many quantum chemistry calculations is to compute the energies of molecules. They are defined as the eigenvalues of the following problem:

$$ H |\psi\rangle = E |\psi\rangle $$

where $H$ is the Hamiltonian of the molecule, $|\psi\rangle$ is an eigenvector of $H$, and $E$ the corresponding eigenenergy. Finding $|\psi\rangle$ and $E$ is a computational task that is, on a classical computer, exponentially costly in the number of chemical orbitals of the Hamiltonian.

The promise of quantum computers is to make this task less costly by using the quantum nature of the computation.

## Variational Quantum Eigensolver
The **Variational Quantum Eigensolver** method solves the following minimization problem :
$$
E = \min_{\vec{\theta}}\; \langle \psi(\vec{\theta}) \,|\, \hat{H} \,|\, \psi(\vec{\theta}) \rangle
$$
where $\hat{H}$ is the molecular Hamiltonian (at fixed geometry) and $|\psi(\vec{\theta})\rangle$ is a parametrized state used to explore the state space.

The role of the **quantum computer** is to construct a trial wavefunction $|\psi(\vec{\theta})\rangle$ and measure its energy $\langle \psi(\vec{\theta}) \,|\, \hat{H} \,|\, \psi(\vec{\theta}) \rangle$. A **classical optimizer** uses this energy to find an optimal value of the angles.

The package ``qat.fermion.chemistry.ucc`` provides tools to construct a particular type of trial wavefunction called "unitary coupled cluster" ansatz, which is known to work well in quantum chemistry.

## Unitary coupled cluster ansatz
The UCC trial state is of the form:
$$
|\psi(\vec{\theta})\rangle = e^{\hat{T}(\vec{\theta}) - \hat{T}^\dagger(\vec{\theta})} |0\rangle
$$
where $\hat{T}(\theta)$ is the *cluster operator*. Its non-truncated form is given by:
$$
\hat{T}(\vec{\theta}) = \hat{T}_1(\vec{\theta}) + \hat{T}_2(\vec{\theta}) + \cdots
$$
where
$$
\hat{T}_1 = \sum_{a\in U}\sum_{i \in O} \theta_a^i\, \hat{a}_a^\dagger \hat{a}_i \qquad
\hat{T}_2 = \sum_{a>b\in U}\sum_{i>j\in O} \theta_{a, b}^{i, j}\, \hat{a}^\dagger_a \hat{a}^\dagger_b \hat{a}_i \hat{a}_j \qquad
\cdots
$$
($O$ is the set of occupied orbitals and $U$, the set of unoccupied ones.)

This notebook will describe how to construct such a UCC ansatz.

## Description of the molecule: dihydrogen
One first has to describe the geometry and atomic content of the molecule.


This is done, in the cell below, using OpenFermion's ``MolecularData`` class, where we specify the chosen basis set and the studied geometry. Here, we chose to study dihydrogen in the so-called STO-3G basis at 0.7414 Angström (internuclear distance.)

In [1]:
from qat.fermion.chemistry.ucc import uccsd
from openfermion.hamiltonians import MolecularData

# Molecule construction:
basis = 'sto-3g'
multiplicity = 1 # Supposed to be set to 1 (i.e. singlet state is assumed!)
bond_length = 0.7414 # In Angstrom.
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]

molecule = MolecularData(geometry, basis, multiplicity)

## Construction of the UCC ansatz
Then this object is given as argument to the function which outputs:
- the parametric state preparation (as a function outputing a QRoutine),
- the initial parameter set,
- the Hamiltonian,
- the number of qubits.

These elements can then be used in a VQE algorithm.

In [2]:
qroutine_uccsd, theta_0, h_spin, nb_qubits = uccsd(molecule)

# Optional parameters are:
#  - The transformation: transformation='bk',
#  - The numerical threshold: threshold=1e-15, 
#  - The active space selection criterion: as_selection=0.001,
#        - either the lower limit value for the NOONs of active orbitals, 
#        - or the list of active orbitals.  
#  - The numbers of single and double excitations to consider: max_nb_single_ex=0, max_nb_double_ex=3, 
#  - The transformation as a BinaryCode, for user-defined ones: code=None, 
#  - The reduction of trivially acted-upon qubits (valid for Braavyi-Kitaev, in certain basis): reduction=False, 
#  - The approximation of double excitation excitations transformed by Bravyi-Kitaev: approx=True.

## VQE execution

### Choice of optimizer

In [3]:
# Definition of the optimizer, e.g. SPSA:
from qat.fermion.optimization import Optimizer
from qat.fermion.spsa import spsa_minimize
spsa_minimizer = Optimizer(spsa_minimize)

### Choice of QPU, and execution

In [4]:
# Definition of the QPU, e.g. LinAlg:
from qat.linalg import get_qpu_server as get_linalg_qpu
my_linalg_qpu = get_linalg_qpu()

from qat.fermion.hamiltonians import IsingHamiltonian
h_vqe = IsingHamiltonian(list_pauli_operators=h_spin.list_pauli_operators,
                         list_pauli_values = h_spin.list_pauli_values)

# Execution of the VQE:
from qat.fermion.vqe import VQE
e, param, nb_evals, energies = VQE(h_vqe,  spsa_minimizer, qroutine_uccsd, theta_0, 
                                   my_linalg_qpu, grouping=False, display=True)

print("VQE energy=", e.real)

Precision reached ( 0.0001 ), iteration number = 8
Energy = (-1.1372701745250948+0j) Optimized parameters = [0.11305897+0.j] 
 Number of function evaluations = 8
VQE energy= -1.1372701745250948


## Adding noise

Later.